## This notebook is just for test.
## This cannot make a complete model because of memory error.
## Thank you.

In [ ]:

import numpy as np 
import pandas as pd 



# 1.前回データの確認

In [ ]:
path = "../input/eda-for-biginner-updated-to-english-ver"

In [ ]:
traindf = pd.read_csv(path+"/traindf.csv")
traindf

# おさらい

In [ ]:
tmp = traindf[traindf["landmark_id"]==7]
tmp

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
for a in tmp["path"]:
    img = cv2.imread(a)
    plt.figure()
    plt.imshow(img)

# おさらい2　import collectionを使って、各idの個数を数えた。それをcount数ごとに並べたのが、dfcnt

In [ ]:
dfcnt = pd.read_csv(path+"/dfcnt.csv")
dfcnt

In [ ]:
plt.scatter(dfcnt["id"],dfcnt["count"])

# この時点で、landmark idの種類は全部で81313個、最小枚数は2枚であることがわかる(前回は138982が6272個に注目してた)ので、
# 各landmark idごとに1枚訓練データ(traindata)、１枚検証データ(validation)にしてpytorchでモデルを作成することを考える。

In [ ]:
traindf

In [ ]:
dfcnt

## dfcntのidでfilteringして、一番上にきたやつをtrain data, 上から2つ目をvalidationとする
## わかりやすくするため、１個で説明

In [ ]:
tmp1 = dfcnt["id"].iloc[0]
tmp1

In [ ]:
tmpdf1 = traindf[traindf["landmark_id"]==tmp1]
tmpdf1

In [ ]:
tlist = []
vlist = []

In [ ]:
tlist.append(tmpdf1.iloc[0].values)
tlist

In [ ]:
vlist.append(tmpdf1.iloc[1].values)

In [ ]:
# これを繰り返す

In [ ]:
from tqdm import tqdm

In [ ]:
import os
os.path.exists("./tdf.csv")

In [ ]:
tlist = []
vlist = []

In [ ]:
if os.path.exists("./tdf.csv")==False:
    

    

    tmp1 = dfcnt["id"].values #.valuesでnumpy. for文はnumpyのほうが早いときがある。

    for a in tqdm(range(len(dfcnt))):

        tmpdf1 = traindf[traindf.landmark_id.values==tmp1[a]]
        tlist.append(tmpdf1.iloc[0].values)
        vlist.append(tmpdf1.iloc[1].values)

In [ ]:
tdf = pd.DataFrame(tlist,columns=tmpdf1.columns)
tdf["repair_id"]=np.arange(0,len(tdf),1)
tdf

In [ ]:
vdf = pd.DataFrame(vlist,columns=tmpdf1.columns)
vdf["repair_id"]=np.arange(0,len(vdf),1)
vdf

In [ ]:
if os.path.exists("./tdf.csv"):
    tdf = pd.read_csv("./tdf.csv")
    vdf = pd.read_csv("./vdf.csv")

In [ ]:
tdf.to_csv("tdf.csv",index=False)
vdf.to_csv("vdf.csv",index=False)

# 全部やっても良いが、自分で作成するときは10枚くらいでテストするほうが効率的

In [ ]:
tdf2 = tdf.iloc[:10,:]
vdf2 = vdf.iloc[:10,:]

In [ ]:
tdf2

In [ ]:
vdf2

# ここからpytorch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import functional as F
from torchvision.models import resnet18
from albumentations import Normalize, Compose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import os
import glob
import multiprocessing as mp



if torch.cuda.is_available():
    device = 'cuda:0'
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = 'cpu'
print(f'Running on device: {device}')

# このサイトがとても分かりやすく書いてくれているので、迷ったらここを見る
https://qiita.com/takurooo/items/e4c91c5d78059f92e76d

# 1. transformの定義

In [ ]:
preprocess = Compose([
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1)
])

# resnextなどのpre-trainモデルは全て、同じ方法で正規化された入力画像を使用しなければならない。それの変換をこの関数で行う。値はdefault。
# Composeは今回あまり、意味をなさない
# https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/albumentations%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81/ に詳細は書いてある

# 2. Dataset

In [ ]:
# 画像をどれだけ小さくするかの処理
ROWS = 32
COLS = 32

In [ ]:
class GLDataset(Dataset):
    
    def __init__(self,img_pass,labels,preprocess=None):
        self.img_pass = img_pass
        self.labels = labels
        self.preprocess = preprocess
        
    def __len__(self):
        return len(self.img_pass)
    
    def __getitem__(self,idx):
        
        # ここからdatasetに食わせる前の前処理の記述。
        
        img_pass = self.img_pass[idx]
        label = self.labels[idx]
        
        land = cv2.imread(img_pass)
        land = cv2.resize(land,(ROWS,COLS),interpolation = cv2.INTER_CUBIC)
        land = cv2.cvtColor(land,cv2.COLOR_BGR2RGB) # augmentを使うときにBGRからRGBにする必要があるのかもしれない。
        
        if self.preprocess is not None: # ここで、前処理を入れてnormalizationしている。
                augmented = self.preprocess(image=land) # preprocessのimageをfaceで読む
                land = augmented['image'] # https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/albumentations%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81/　に書いてある
                
        return {'landmarks': land.transpose(2, 0, 1), 'label': np.array(label, dtype=int)}  # pytorchはchannnl, x, yの形。これは辞書型で返している。(扱いやすいというだけかも。)
        
        
        
        
        
        
        
        

## 1つ1つ追って、何やっているかを見ていく。

In [ ]:
land = cv2.imread(tdf2["path"].iloc[0])

In [ ]:
plt.imshow(land)

In [ ]:
land = cv2.resize(land,(ROWS,COLS),interpolation = cv2.INTER_CUBIC)

In [ ]:
plt.imshow(land)

In [ ]:
land = cv2.cvtColor(land,cv2.COLOR_BGR2RGB) # augmentを使うときにBGRからRGBにする必要があるのかもしれない。

In [ ]:
plt.imshow(land)

In [ ]:
augmented = preprocess(image=land) # preprocessのimageをfaceで読む
land = augmented['image']

In [ ]:
plt.imshow(land)

In [ ]:
land.shape

In [ ]:
land=land.transpose(2, 0, 1)

In [ ]:
land.shape

# Datasetのinstance化

In [ ]:
# instance化
train_dataset = GLDataset(
    img_pass=tdf2["path"],
    labels=tdf2["repair_id"].to_numpy(),
    preprocess=preprocess
)

val_dataset = GLDataset(
    img_pass=vdf2["path"],
    labels=vdf2["repair_id"].to_numpy(),
    preprocess=preprocess
)

In [ ]:
print(train_dataset[0])

# 3. DataLoader

In [ ]:
BATCH_SIZE = 2

#NUM_WORKERS = mp.cpu_count()
NUM_WORKERS = mp.cpu_count() # ここを0にしないと動かない。cpuの仕様個数。←実は動くことが判明。classの中身次第！

In [ ]:
#NUM_WORKERS = mp.cpu_count()
#NUM_WORKERS

In [ ]:
## DataLoaderはimport torch.utils.data.Datasetでimport済みのもの
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, #https://schemer1341.hatenablog.com/entry/2019/01/06/024605 を参考. idがわからなくなる
    num_workers=NUM_WORKERS
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

# reference) class化してすっきりする

In [ ]:
class PytorchDataSet():
    
    def __init__(self,Dataset,train_imgpath,train_label,val_imgpath,val_label,batch_size,num_workers,shuffle=True):
        
        self.Dataset = Dataset
        self.train_imgpath = train_imgpath
        self.train_label = train_label
        self.val_imgpath = val_imgpath
        self.val_label = val_label
        
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.shuffle = shuffle
               
               
        self.train_dataset = self.Dataset(
            img_pass=self.train_imgpath,
            labels=self.train_label.to_numpy(),
            preprocess=preprocess
)
        
        self.val_dataset = self.Dataset(
            img_pass=self.val_imgpath,
            labels=self.val_label.to_numpy(),
            preprocess=preprocess
        )
        
                ## DataLoaderはimport torch.utils.data.Datasetでimport済みのもの
        self.train_dataloader = DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=self.shuffle, #https://schemer1341.hatenablog.com/entry/2019/01/06/024605 を参考. idがわからなくなる
            num_workers=self.num_workers
        )
        self.val_dataloader = DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            num_workers=self.num_workers
        )
        

In [ ]:
# example
base = PytorchDataSet(GLDataset,tdf2["path"],tdf2["repair_id"],vdf2["path"],vdf2["repair_id"],BATCH_SIZE,NUM_WORKERS)

In [ ]:
# こうしておくと、例えば、全部の画像でやるときに便利。
# 全部の画像でやるとき
# base = PytorchDataSet(GLDataset,tdf["path"],tdf["repair_id"],vdf["path"],vdf["repair_id"],BATCH_SIZE,NUM_WORKERS)

In [ ]:
# example
for a in base.train_dataloader:
    print(a)

# 初日はここまでかな・・・

# ここからDeep learningの設定

In [ ]:
encoder = resnet18(pretrained=True) #  pretrained = Trueはimagenetからpre-trainモデルを使う

In [ ]:
class LandmarkClassifier(nn.Module): # nn.Moduleが入っているのが、モデルの定義っぽい
    
    def __init__(self, encoder, in_channels=3, num_classes=len(dfcnt)): 
        
        super(LandmarkClassifier, self).__init__() # nn.Moduleの__init__を継承。https://blog.codecamp.jp/python-class-2
        
        self.encoder = encoder
        
        # Modify input layer. # 入口のチャンネル数を合わせる。defaultのResnetのclassは64 channelになっているので、それをin_channelsにする。
        # ここの記述がencoderごとに代わるので、efficientnet使うときは変えなければいけない
        
        self.encoder.conv1 = nn.Conv2d(
            in_channels,
            64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        
        # Modify output layer.# 出口の個数も合わせる。defaultのResnetの出口は、1000になっているので、num_classesに変更
        # ここの記述がencoderごとに代わるので、efficientnet使うときは変えなければいけない
        
        self.encoder.fc = nn.Linear(512 * 1, num_classes)

    def forward(self, x): # 呼び出されたときに、xの引数があると、sigmoidで返す。ex) 後ほどのclassifier(sample_batched["landmark"]) みたいなところ。
        
        # sigmoidで返すときは以下の感じ　https://aidiary.hatenablog.com/entry/20180203/1517629555
        # return torch.sigmoid(self.encoder(x))
        
        # 多値問題でも、softmaxはここでは使わない。nn.CrossEntropyに既に入っているため
        return self.encoder(x)
    
    
    
    
    
    ### 以下、どこのパラメーターを最適化するか。簡易テスト用は真ん中。フルオプションは下。ここはご参考。###
    
    def freeze_all_layers(self):# 中間層のパラメーター(重みづけ)を全部変えない。
        for param in self.encoder.parameters():
            param.requires_grad = False

    def freeze_middle_layers(self):
        self.freeze_all_layers()
        
        for param in self.encoder.conv1.parameters():# 最初のパラメーター(重みづけ)を変える。
            param.requires_grad = True
            
        for param in self.encoder.fc.parameters():# 最後の(重みづけ)を変える。
            param.requires_grad = True

    def unfreeze_all_layers(self):# 中間層のパラメーター(重みづけ)を全部変える。
        for param in self.encoder.parameters():
            param.requires_grad = True

In [ ]:

classifier = LandmarkClassifier(encoder=encoder, in_channels=3, num_classes=len(dfcnt)) # classifierはDeepfakeClassifierのインスタンス化

# classifier.to(device) # ここがKerasとは違うところ。GPUに送りますよーという意味。今回はcpuなので、pass.

classifier.train() # ここがKerasとは違うところ。 訓練モードの場合 classifier.train() , 推論の場合 classifier.eval() と書く。Normalizeのプロセスなどが違うらしい。

In [ ]:
# 今回は全部最適化
classifier.unfreeze_all_layers()

In [ ]:
criterion = nn.CrossEntropyLoss()
# 多値分類はcrossentropy

In [ ]:
#optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()),lr=1e-5)
optimizer = optim.Adam(classifier.parameters(),lr=1e-6)
#optimizer = optim.SGD(classifier.parameters(),lr=1e-5)

# conv層(model.features)のパラメータを固定し、全結合層(model.classifier)のみをfinetuneする設定です。
# 全層のうち、requires_gradがTrueの層のみをfinetuneするという意味です。
# lr : 学習率

## 効率的なやり方
## Process 1 : Simpleにしてできるところからやる。例えば1つだけやる
## Process 2 : いったんまとめてみる
## Process 3 : for文にして回してみる
## Process 4 : 汎用性を持たせる (数字 → 文字化)
## Process 5 : さらに汎用性を持たせる (関数化)
## Process 6 : class化する

# モデルの最適化は複雑なので、1epochごとに、1つ1つ見ていく
# まずはtrain

In [ ]:
classifier.train()

In [ ]:
train_dataloader[0]

In [ ]:
# 読めないので、以下で確認。
for a in train_dataloader:
    print(a)


In [ ]:
for a in train_dataloader:
    y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
    print(y_pred)
    

In [ ]:
y_pred

In [ ]:
len(y_pred[0])

# lossを算出 (crossentropyloss)

In [ ]:
for a in train_dataloader:
    y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
    label = a["label"]
    
    loss = criterion(y_pred, label)
    print(loss)

In [ ]:
for a in train_dataloader:
    y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
    label = a["label"]
    
    loss = criterion(y_pred, label)
    print(loss.item())

In [ ]:
for a in train_dataloader:
    y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
    label = a["label"]
    
    loss = criterion(y_pred, label)
    print(loss.item())
    
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad() # おきまり。初期化。
    loss.backward() # 後方にバック
    optimizer.step() # 重みづけの最適化

# このあと、epochごとにまとめることを考えると、loss.itemの一番最後を保存。

In [ ]:
classifier.train()
trainloss = []

for a in train_dataloader:
    y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
    
    
    label2 = a["label"]
    
    loss = criterion(y_pred, label2)
    print(loss.item()) # itemは0次元tensorから整数を出す場合。
    
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad() # おきまり。初期化。
    loss.backward() # 後方にバック
    optimizer.step() # 重みづけの最適化
    
trainloss.append(loss.item())

# 関数化

In [ ]:
def trainmodel(train_dataloader):
    classifier.train()
    
    for a in train_dataloader:
        y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
        label = a["label"]

        loss = criterion(y_pred, label)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() # おきまり。初期化。
        loss.backward() # 後方にバック
        optimizer.step() # 重みづけの最適化
        
    return(loss.item())
    
    

# validationを同様に作る。

In [ ]:
classifier.eval() # ここが推論なので、変えなきゃダメ。

In [ ]:
classifier.eval() # ここが推論なので、変えなきゃダメ。
val_loss = []

for a in val_dataloader:
    y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
    
   
    label = a["label"]
    
    lossval = criterion(y_pred, label)
    print(lossval.item()) # itemは0次元tensorから整数を出す場合。

val_loss.append(lossval.item())
    


# 関数化

In [ ]:
def valmodel(val_dataloader):
    
    classifier.eval()
    
    for a in val_dataloader:

        y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
        label = a["label"]

        loss = criterion(y_pred, label)
        
    return(loss.item())
    
    

# くっつけてepochを回す

In [ ]:
epochs = 10

In [ ]:
"""

trainloss = []
valloss = []


for epoch in range(epochs):
    
    tloss_tmp =[]
    vloss_tmp =[]

    
    ####### train #######
    classifier.train()
    

    for a in train_dataloader:
        y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
        
        # y_pred = torch.sigmoid(y_pred) 本来違うのだけどなぜかこれを入れた方がスコアが良い

        y_pred2 = y_pred.squeeze(dim=-1)
        label2 = a["label"].squeeze(dim=-1)

        loss = criterion(y_pred2, label2)
        tloss_tmp.append(loss.item())
        # print(loss.item()) # itemは0次元tensorから整数を出す場合。

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() # おきまり。初期化。
        loss.backward() # 後方にバック
        optimizer.step() # 重みづけの最適化

    trainloss.append(tloss_tmp[-1])
    
    #######validation#######
    
    classifier.eval() # ここが推論なので、変えなきゃダメ。
    

    for a in val_dataloader:
        y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
        
        # y_pred = torch.sigmoid(y_pred) 本来違うのだけどなぜかこれを入れた方がスコアが良い


        y_pred2 = y_pred.squeeze(dim=-1)
        label2 = a["label"].squeeze(dim=-1)

        lossval = criterion(y_pred2, label2)
        vloss_tmp.append(lossval.item())
        # print(lossval.item()) # itemは0次元tensorから整数を出す場合。

    valloss.append(vloss_tmp[-1])
    
    print(str(epoch) + "_end")

    

"""

# 関数化した版

In [ ]:
trainloss = []
valloss = []


for epoch in range(epochs):
    
    trainloss.append(trainmodel(train_dataloader))
    valloss.append(valmodel(val_dataloader))
    
    print(str(epoch) + "_end")

    

In [ ]:
x = np.arange(epochs)
plt.scatter(x,trainloss)

In [ ]:
plt.figure()
plt.scatter(x,valloss)

# validationlossが下がったら、モデルをsaveする記述を追加。

In [ ]:
savename="resnet18.pth"

In [ ]:
trainloss = []
valloss = []

bestloss = None

for epoch in range(epochs):
    
    trainloss.append(trainmodel(train_dataloader))
    valloss.append(valmodel(val_dataloader))
    
    print(str(epoch) + "_end")
    
    #######modelをsave#######
    
    if bestloss is None:
        bestloss = valloss[-1]
        state = {
                'state_dict': classifier.state_dict(),
                'optimizer_dict': optimizer.state_dict(),
                "bestloss":bestloss,
            }

        torch.save(state, savename)
        
        print("save the first model")
    
    elif valloss[-1] < bestloss:
        
        bestloss = valloss[-1]
        state = {
                'state_dict': classifier.state_dict(),
                'optimizer_dict': optimizer.state_dict(),
            "bestloss":bestloss,
            }

        torch.save(state, savename)
        
        print("found a better point")
    
    else:
        pass

    

# 関数化

In [ ]:
def savemodel(bestloss,valloss):
    
    #######modelをsave#######
    
    if bestloss is None:
        bestloss = valloss[-1]
        state = {
                'state_dict': classifier.state_dict(),
                'optimizer_dict': optimizer.state_dict(),
                "bestloss":bestloss,
            }

        torch.save(state, savename)
        
        print("save the first model")
    
    elif valloss[-1] < bestloss:
        
        bestloss = valloss[-1]
        state = {
                'state_dict': classifier.state_dict(),
                'optimizer_dict': optimizer.state_dict(),
                "bestloss":bestloss,
            }

        torch.save(state, savename)
        
        print("found a better point")
    
    else:
        pass
    
    return bestloss


In [ ]:
trainloss = []
valloss = []

bestloss = None

for epoch in range(epochs):
    
    trainloss.append(trainmodel(train_dataloader))
    valloss.append(valmodel(val_dataloader))
    
    print(str(epoch) + "_end")
    
    bestloss= savemodel(bestloss,valloss)
    
    print(bestloss)
    
    
    

In [ ]:
plt.scatter(x,trainloss)
plt.scatter(x,valloss)

# Reference : ここまでをclass化してすっきりする→汎用性をもたせる

In [ ]:
class MakingModel():
    
    def __init__(self,train_dataloader,val_dataloader,epochs,savename,bestloss=None):
        
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.epochs = epochs
        
        if bestloss is None:
            self.bestloss = bestloss
        else:
            self.bestloss = None
            
        self.savename = savename

        self.trainloss=[]
        self.valloss=[]
        
        self.makemodel()
        
    
    
    def trainmodel(self):
        
        classifier.train()

        for a in self.train_dataloader:
            y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
            label = a["label"]

            loss = criterion(y_pred, label)

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad() # おきまり。初期化。
            loss.backward() # 後方にバック
            optimizer.step() # 重みづけの最適化

        return(loss.item())
    
    
    def valmodel(self):
        classifier.eval()

        for a in self.val_dataloader:
            y_pred = classifier(a['landmarks']) # onenoteでいうoutput = model(train_x)
            label = a["label"]

            loss = criterion(y_pred, label)


        return(loss.item())
    
    
    
    def savemodel(self):
    
        #######modelをsave#######

        if self.bestloss is None:
            self.bestloss = self.valloss[-1]
            state = {
                    'state_dict': classifier.state_dict(),
                    'optimizer_dict': optimizer.state_dict(),
                    "bestloss":self.bestloss,
                }

            torch.save(state, self.savename)

            print("save the first model")

        elif self.valloss[-1] < self.bestloss:

            self.bestloss = self.valloss[-1]
            state = {
                    'state_dict': classifier.state_dict(),
                    'optimizer_dict': optimizer.state_dict(),
                    "bestloss":self.bestloss,
                }

            torch.save(state, self.savename)

            print("found a better point")

        else:
            pass

        return self.bestloss
    
    
    def makemodel(self):

        for epoch in range(self.epochs):

            self.trainloss.append(self.trainmodel())
            self.valloss.append(self.valmodel())

            print(str(epoch) + "_end")

            self.bestloss= self.savemodel()

            print(self.bestloss)




    
    
        
        

In [ ]:
tmp = MakingModel(train_dataloader,val_dataloader,epochs,"test.pth",bestloss=None)

In [ ]:
x = np.arange(tmp.epochs)
plt.scatter(x,tmp.trainloss)
plt.scatter(x,tmp.valloss)

In [ ]:
tmp.savename

# ここからは推論

# 基本は、model作成と同じだが、ラベルがないので、それようのclassを作る

testのpassを出す

In [ ]:
test_path = []

import os
for dirname, _, filenames in os.walk('../input/landmark-recognition-2020/test'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        test_path.append(os.path.join(dirname, filename))

In [ ]:
test_path[:10]

In [ ]:
testid = [s.split("/")[-1] for s in test_path]
testid = [s.split(".")[0] for s in testid]
testid[:10]

In [ ]:
testdf = pd.DataFrame()
testdf["id"] = testid
testdf["path"] = test_path
testdf

In [ ]:
class GLDataset_inf(Dataset):
    
    def __init__(self,testid,img_pass,preprocess=None):
        self.testid = testid
        self.img_pass = img_pass
        self.preprocess = preprocess
        
    def __len__(self):
        return len(self.img_pass)
    
    def __getitem__(self,idx):
        
        # ここからdatasetに食わせる前の前処理の記述。
        
        img_id = self.testid[idx]
        
        img_pass = self.img_pass[idx]
        
        land = cv2.imread(img_pass)
        land = cv2.resize(land,(ROWS,COLS),interpolation = cv2.INTER_CUBIC)
        land = cv2.cvtColor(land,cv2.COLOR_BGR2RGB) # augmentを使うときにBGRからRGBにする必要があるのかもしれない。
        
        if self.preprocess is not None: # ここで、前処理を入れてnormalizationしている。
                augmented = self.preprocess(image=land) # preprocessのimageをfaceで読む
                land = augmented['image'] # https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/albumentations%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81/　に書いてある
                
        return {"id":img_id,'landmarks': land.transpose(2, 0, 1)}  # pytorchはchannnl, x, yの形。これは辞書型で返している。(扱いやすいというだけかも。)
        
        
        
        
        
        
        
        

In [ ]:
testdf2 = testdf.iloc[:10,:]

# インスタンス化(とりあえず10枚)

In [ ]:
test_dataset = GLDataset_inf(
    testdf2["id"],
    testdf2["path"],
    preprocess=preprocess
)

# Dataloader化

In [ ]:
## DataLoaderはimport torch.utils.data.Datasetでimport済みのもの
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, #https://schemer1341.hatenablog.com/entry/2019/01/06/024605 を参考. idがわからなくなる
    num_workers=0
)

# torchモデルのload ※ おそらく必要ないが、notebookわけたときように。

In [ ]:
MODEL_PATH = "resnet18.pth"

encoder = resnet18(pretrained=False) # 学習済みモデルをインターネットからひっぱるわけじゃないので、false
classifier = LandmarkClassifier(encoder=encoder, in_channels=3, num_classes=len(dfcnt)) # classifierはDeepfakeClassifierのインスタンス化

# classifier.to(device) # ここがKerasとは違うところ。GPUに送りますよーという意味

state = torch.load(MODEL_PATH, map_location=lambda storage, loc: storage)
classifier.load_state_dict(state['state_dict'])

classifier.eval() # 推論なので、eval

In [ ]:
presub = []

for a in tqdm(test_dataloader):
    y_pred = classifier(a['landmarks'])
    cnt = F.softmax(y_pred).cpu().detach().numpy()
    
    y_pred = y_pred.cpu().detach().numpy()
    
    
    for b in range(len(y_pred)):
        tmp = np.argmax(y_pred[b])
        conf = cnt[b][tmp]
        
        presub.append([a["id"][b],tmp,conf])
    
    
    
        

In [ ]:
presubdf = pd.DataFrame(presub,columns=["id","landid","conf"])

In [ ]:
presubdf

#全テストデータで実行

In [ ]:
test_dataset = GLDataset_inf(
    testdf["id"],
    testdf["path"],
    preprocess=preprocess
)

## DataLoaderはimport torch.utils.data.Datasetでimport済みのもの
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, #https://schemer1341.hatenablog.com/entry/2019/01/06/024605 を参考. idがわからなくなる
    num_workers=NUM_WORKERS
)

In [ ]:
presub = []

for a in tqdm(test_dataloader):
    y_pred = classifier(a['landmarks'])
    cnt = F.softmax(y_pred).cpu().detach().numpy()
    
    y_pred = y_pred.cpu().detach().numpy()
    
    
    for b in range(len(y_pred)):
        tmp = np.argmax(y_pred[b])
        conf = cnt[b][tmp]
        
        presub.append([a["id"][b],tmp,conf])
    

       

In [ ]:
presubdf = pd.DataFrame(presub,columns=["testid","repair_id","conf"])
presubdf

In [ ]:
tdf

# repairidの補正

In [ ]:
mdf = pd.merge(presubdf,tdf,on="repair_id",how="left")
mdf

In [ ]:
sub = []

for a in range(len(mdf)):
    
    tmp = str(mdf["landmark_id"].iloc[a]) + " " + str(mdf["conf"].iloc[a])
    sub.append(tmp)

mdf["submission"]=sub
    

In [ ]:
submissiondf = mdf[["testid","submission"]]
submissiondf

In [ ]:
submissiondf.columns=["id","landmarks_pytorch"]
submissiondf

In [ ]:
# sample submissionの形式に。

In [ ]:
sample = pd.read_csv("../input/landmark-recognition-2020/sample_submission.csv")

In [ ]:
sample

In [ ]:
submission = pd.merge(sample,submissiondf,on="id",how="left")
submission

In [ ]:
submission["landmarks"] = submission["landmarks_pytorch"]
submission

In [ ]:
submission = submission.iloc[:,:2]
submission

In [ ]:
submission.to_csv("submission.csv")

# memo : メモリが足りなくて、全部学習できない。
# memo2 : 学習済みresnet 18はinternet onだと実行できない。→　やり方はあるが、
# モデル構築と、推論はnotebookを分けてもよい。